What you need:
* The tablet or phone should be connected to the computer with debug mode enabled and an adbd server running and connected to the tablet

[Enabling adb Debugging](http://developer.android.com/tools/help/adb.html#Enabling)

In [14]:
# Check if we can talk to the tablet
!tools/adb devices

List of devices attached 
554794d3444cc8b0708	device



When our device shows up in the list of devices we should be good to go.  The rest of these instructions assume this is set up and also that only one device is attached.

In [6]:
%%bash
# Lets note what partitions we have on the device

tools/adb shell "cat /proc/partitions" > device_info/partitions.proc.info
cat device_info/partitions.proc.info

major minor  #blocks  name

  93        0      32768 nanda
  93        8      16384 nandb
  93       16      16384 nandc
  93       24     786432 nandd
  93       32    1048576 nande
  93       40      16384 nandf
  93       48      32768 nandg
  93       56     131072 nandh
  93       64     131072 nandi
  93       72    4675584 nandj


In [7]:
%%bash
# Lets note the nameing conventions in the device

tools/adb shell "ls -al /dev/block/by-name" > device_info/by-name.block.dev.info
cat device_info/by-name.block.dev.info

lrwxrwxrwx root     root              2015-03-14 14:17 UDISK -> /dev/block/nandj
lrwxrwxrwx root     root              2015-03-14 14:17 boot -> /dev/block/nandc
lrwxrwxrwx root     root              2015-03-14 14:17 boot-resource -> /dev/block/nanda
lrwxrwxrwx root     root              2015-03-14 14:17 cache -> /dev/block/nandh
lrwxrwxrwx root     root              2015-03-14 14:17 data -> /dev/block/nande
lrwxrwxrwx root     root              2015-03-14 14:17 databk -> /dev/block/nandi
lrwxrwxrwx root     root              2015-03-14 14:17 env -> /dev/block/nandb
lrwxrwxrwx root     root              2015-03-14 14:17 misc -> /dev/block/nandf
lrwxrwxrwx root     root              2015-03-14 14:17 recovery -> /dev/block/nandg
lrwxrwxrwx root     root              2015-03-14 14:17 system -> /dev/block/nandd


I've used a method below to dump our partitions straight to our local file system.  Because the adb shell acts as a terminal it doesn't handle binary data very well when piping the out put (creates a few crlf linefeed stuff which doesn't help our data).  Fortunately the device i'm using did have busybox installed which has a handy tool for sending binary data. Here is the link to the article which may explain this more...
[Read binary stdout data from adb shell?](http://stackoverflow.com/questions/13578416/read-binary-stdout-data-from-adb-shell)

In [1]:
# Now lets pull each partition into our tablet directory only pull one at a time

!tools/adb shell "busybox stty -onlcr; dd if=/dev/block/nanda" > tablet/nanda.boot-resource.img

In [2]:
!tools/adb shell "busybox stty -onlcr; dd if=/dev/block/nandb" > tablet/nandb.env.img

In [3]:
!tools/adb shell "busybox stty -onlcr; dd if=/dev/block/nandc" > tablet/nandc.boot.img

In [4]:
!tools/adb shell "busybox stty -onlcr; dd if=/dev/block/nandd" > tablet/nandd.system.img

In [11]:
# Optional you may not need this if you plan on clearing all data

#!tools/adb shell "busybox stty -onlcr; dd if=/dev/block/nande" > tablet/nande.data.img

In [8]:
!tools/adb shell "busybox stty -onlcr; dd if=/dev/block/nandf" > tablet/nandf.misc.img

In [5]:
!tools/adb shell "busybox stty -onlcr; dd if=/dev/block/nandg" > tablet/nandg.recovery.img

In [9]:
# Optional would probably want to just clear this

#!tools/adb shell "busybox stty -onlcr; dd if=/dev/block/nandh" > tablet/nandh.cache.img

In [10]:
# Probably optional as well, may hold a tar.gz backup file

!tools/adb shell "busybox stty -onlcr; dd if=/dev/block/nandi" > tablet/nandi.databk.img

In [12]:
# Optional this is the SD-Card which can be 4GB upwards depends if you want to back it up

#!tools/adb shell "busybox stty -onlcr; dd if=/dev/block/nandj" > tablet/nandj.UDISK.img

In [15]:
# We should now have a backup or each partition
!ls -lGg tablet/*.img

-rwxrwxrwx 1  33554532 Mar 16 12:35 tablet/nanda.boot-resource.img
-rwxrwxrwx 1  16777316 Mar 16 12:37 tablet/nandb.env.img
-rwxrwxrwx 1  16777316 Mar 16 12:37 tablet/nandc.boot.img
-rwxrwxrwx 1 805306475 Mar 16 12:42 tablet/nandd.system.img
-rwxrwxrwx 1  16777316 Mar 16 13:04 tablet/nandf.misc.img
-rwxrwxrwx 1  33554532 Mar 16 12:43 tablet/nandg.recovery.img
-rwxrwxrwx 1 134217832 Mar 16 13:06 tablet/nandi.databk.img


In [16]:
# Lets archive what we've got and can refer to the originals without having to dump again
!tar zcvf org/Allwinner_Polaris_A70H.tar.gz tablet/*.img

tablet/nanda.boot-resource.img
tablet/nandb.env.img
tablet/nandc.boot.img
tablet/nandd.system.img
tablet/nandf.misc.img
tablet/nandg.recovery.img
tablet/nandi.databk.img


Now lets investigate what we've got...